In [1]:
# Installing Chrome Driver and Selenium in Google Colab
!pip install chromedriver-autoinstaller
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 28.6 MB/s eta 0:00:00


In [2]:
# Importing the required libraries
import requests
import pandas as pd
import re
from google.colab import drive
drive.mount('/content/drive/')

import time

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException

Mounted at /content/drive/


In [97]:
page_number = 0
response_str = ""
condition_bool = True
while(condition_bool):
  page_number += 1
  url = "https://vrdb.app/games?page=" + str(page_number)

  # Send the GET request
  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for non-2xx status codes
  except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    # For human-readable content, use response.text
    # For raw data, use response.content
  pattern_script = r'<script>__sveltekit_1pr461f.resolve(.*?)</script>'
  match_script = re.findall(pattern_script, str(response.content))
  response_str += match_script[0]
  # To end the execution of the while loop
  condition_bool = "data:[]" not in str(response.content)

del url, condition_bool

pattern_name = r'name:"(.*?)"'
pattern_rating_count = r'rating_count:(.*?),'
pattern_genre = r'genres:\["(.*?)"\]'
pattern_links = r'store_link:"(.*?)"'
pattern_currency_USD = r'price:{currency:"USD",amount:(.*?),'
pattern_game_mode = r'game_mode:"(.*?)"'
pattern_supported_player_modes = r'supported_player_modes:\[(.*?)\]'
pattern_supported_controllers = r'supported_controllers:\["(.*?)"\]'
pattern_platforms = r'platforms:\[(.*?)\]'
pattern_languages = r'languages:\["(.*?)"\]'
pattern_release_date_short = r'release_date_short:"(.*?)"'
pattern_rating_score = r'rating_score:(.*?),'
pattern_description = r'description:"(.*?)",'
pattern_category = r'category:"(.*?)"'
pattern_age_rating = r'age_rating:"(.*?)"'

match_name = re.findall(pattern_name, response_str)
match_rating_count = re.findall(pattern_rating_count, response_str)
match_genre = re.findall(pattern_genre, response_str)
match_links = re.findall(pattern_links, response_str)
match_currency_USD = re.findall(pattern_currency_USD, response_str)
match_game_mode = re.findall(pattern_game_mode, response_str)
match_supported_player_modes = re.findall(pattern_supported_player_modes, response_str)
match_supported_controllers = re.findall(pattern_supported_controllers, response_str)
match_platforms = re.findall(pattern_platforms, response_str)
match_languages = re.findall(pattern_languages, response_str)
match_release_date_short = re.findall(pattern_release_date_short, response_str)
match_rating_score = re.findall(pattern_rating_score, response_str)
match_description = re.findall(pattern_description, response_str)
match_category = re.findall(pattern_category, response_str)
match_age_rating = re.findall(pattern_age_rating, response_str)

# Deleting the response from the URL
del response, response_str

# Deleting the temp pattern variables
del pattern_name, pattern_rating_count, pattern_genre, pattern_links, \
pattern_currency_USD, pattern_game_mode, pattern_supported_player_modes, \
pattern_supported_controllers, pattern_platforms, pattern_languages, \
pattern_release_date_short, pattern_rating_score, pattern_description, \
pattern_category, pattern_age_rating, pattern_script

# Cleaning the data
del match_rating_count[::2], match_rating_score[::2]

for counter in range(len(match_name)):
  match_name[counter] = match_name[counter].replace("\\'","'")
  match_links[counter] = match_links[counter].replace('?utm_source=vrdb.app','')
  # Cleaning the lists in the data
  match_genre[counter] = match_genre[counter].replace('"','')
  match_platforms[counter] = match_platforms[counter].replace('"','')
  match_supported_player_modes[counter] = match_supported_player_modes[counter].replace('"','')
  match_supported_controllers[counter] = match_supported_controllers[counter].replace('"','')
  match_languages[counter] = match_languages[counter].replace('"','')
  # Cleaning the currency data
  match_currency_USD[counter] = match_currency_USD[counter].replace('null','0')
  match_currency_USD[counter] = str(float(match_currency_USD[counter]) / 100)

# Deleting the temp pattern variable
del counter

# Writing the dictionary (data) to Excel file
dict_csv = {"Name":match_name, "Genre":match_genre, \
            "Meta Link":match_links, "Release Date":match_release_date_short, \
            "Rating Score":match_rating_score, "Rating Count":match_rating_count, \
             "Supported Platforms":match_platforms, "Game Mode":match_game_mode, \
            "Supported Player Modes":match_supported_player_modes, \
            "Supported Controllers":match_supported_controllers, \
            "Languages":match_languages, "Price (USD)":match_currency_USD, \
            "Description":match_description, "Category":match_category, \
            "Age Rating":match_age_rating}
df = pd.DataFrame(dict_csv)
df.to_excel("/content/drive/My Drive/Meta Quest Games.xlsx", index=False)